The purpose of this notebook is to apply the support vector machines model to RNA Seq Data. The data comes fro the study by Kopp et al. The experiment aimed to determine the effectiveness of a medication, Lumacaftor/ivacaftor, for Cystic Fibrosis. The experiment involved a total of 40 patients. 20 healthy control and 20 with Cystic Fibrosis. The 20 with CF were also treated with a medication Lumacaftor/ivacaftor for a total of 60 samples.

In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns #For visualizatin
%matplotlib inline
from sklearn.metrics import classification_report,confusion_matrix #For analysis after model application
from sklearn.model_selection import train_test_split #To train SVM model
from sklearn.model_selection import GridSearchCV #To change parameters in the SVM 

In [42]:
data_df = pd.read_excel ('/Users/danielaquijano/Documents/GitHub/Machine-Learning-Course-Projects/sourcefiles/GSE124548_RNAseq.xlsx')
data_df.head()

,RowID,ID,Description,EntrezID,Class,pre_drug.vs.control_ShrunkenLog2FC,pre_drug.vs.control_MLELog2FC,pre_drug.vs.control_pVal,pre_drug.vs.control_pAdj,post_drug.vs.control_ShrunkenLog2FC,...,Raw_Orkambi_021_Base,Raw_Orkambi_021_V2,Raw_Orkambi_022_Base,Raw_Orkambi_022_V2,Raw_Orkambi_024_Base,Raw_Orkambi_024_V2,Raw_Orkambi_025_Base,Raw_Orkambi_025_V2,Raw_Orkambi_026_Base,Raw_Orkambi_026_V2
0,1,A1BG,alpha-1-B glycoprotein,1,protein_coding,-0.050497,-0.066327,0.691693,0.824882,0.082631,...,113,89,31,125,44,78,113,98,38,70
1,2,A1BG-AS1,A1BG antisense RNA 1,503538,lncRNA,0.004178,0.032895,0.971929,0.986309,0.155902,...,40,36,26,87,23,72,74,55,64,53
2,3,A2M-AS1,A2M antisense RNA 1 (head to head),144571,lncRNA,0.356337,0.932311,0.050847,0.179776,-0.359549,...,16,23,40,104,370,199,81,30,228,33
3,4,AAAS,aladin WD repeat nucleoporin,8086,protein_coding,-0.139005,-0.152091,0.285215,0.495514,-0.118870,...,130,71,28,178,62,219,176,106,160,121
4,5,AACS,acetoacetyl-CoA synthetase,65985,protein_coding,0.138032,0.143003,0.088856,0.251817,0.075925,...,192,157,71,228,130,185,198,114,131,157


Possible comparisons to perform:

An upaired analysis comparing healthy vs cystic fibrosis. These will identify genes that can distinguish a healthy patient from one with CF.

A paired analysis comparing the CF patients before and after treatment. This will give us an idea of how the CF patients are responding to the treatment. However we expect this to be noisy since the motivation for this experiment was that not all patients are responding similarly.

Expression data from the top 10 most differentially expressed genes will be used for unpaired analysis.

First test how well it can perform in predicting Healthy and CF patients, and then we will pretend the CF treated is our test set and make our prediction.

In [43]:
#Trim dataframe to only contain 60 columns
#Get column names/dataset information
data_df.info()
for col in data_df.columns:
    print(col)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15570 entries, 0 to 15569
Columns: 140 entries, RowID to Raw_Orkambi_026_V2
dtypes: float64(75), int64(62), object(3)
memory usage: 16.6+ MB
RowID
ID
Description
EntrezID
Class
pre_drug.vs.control_ShrunkenLog2FC
pre_drug.vs.control_MLELog2FC
pre_drug.vs.control_pVal
pre_drug.vs.control_pAdj
post_drug.vs.control_ShrunkenLog2FC
post_drug.vs.control_MLELog2FC
post_drug.vs.control_pVal
post_drug.vs.control_pAdj
post_drug.vs.pre_drug_ShrunkenLog2FC
post_drug.vs.pre_drug_MLELog2FC
post_drug.vs.pre_drug_pVal
post_drug.vs.pre_drug_pAdj
control_Mean
pre-drug_Mean
post-drug_Mean
Norm_10_HC_Auto_066_237
Norm_11_Orkambi_006_Base
Norm_12_Orkambi_007_V2
Norm_13_HC_Auto_068_239
Norm_14_Orkambi_007_Base
Norm_15_Orkambi_009_V2
Norm_16_HC_Auto_072_243
Norm_17_Orkambi_009_Base
Norm_18_Orkambi_010_V2
Norm_19_HC_Auto_074_245
Norm_1_Orkambi_001_Base
Norm_20_Orkambi_010_Base
Norm_21_HC_Immune_004
Norm_22_Orkambi_012_V2
Norm_23_HC_Auto_076_247
Norm_24_Orkambi_

In [44]:
#Obtain column numbers to extract 60 column raw data
print(data_df.columns.get_loc("Raw_10_HC_Auto_066_237"))
print(data_df.columns.get_loc('Raw_Orkambi_026_V2'))

80
139


In [63]:
#Slice Dataframe to only include raw data
raw_data=data_df.iloc[:,80:140]
raw_data.head()

,Raw_10_HC_Auto_066_237,Raw_11_Orkambi_006_Base,Raw_12_Orkambi_007_V2,Raw_13_HC_Auto_068_239,Raw_14_Orkambi_007_Base,Raw_15_Orkambi_009_V2,Raw_16_HC_Auto_072_243,Raw_17_Orkambi_009_Base,Raw_18_Orkambi_010_V2,Raw_19_HC_Auto_074_245,...,Raw_Orkambi_021_Base,Raw_Orkambi_021_V2,Raw_Orkambi_022_Base,Raw_Orkambi_022_V2,Raw_Orkambi_024_Base,Raw_Orkambi_024_V2,Raw_Orkambi_025_Base,Raw_Orkambi_025_V2,Raw_Orkambi_026_Base,Raw_Orkambi_026_V2
0,81,38,112,46,64,196,125,140,72,104,...,113,89,31,125,44,78,113,98,38,70
1,49,31,84,26,38,86,80,76,38,56,...,40,36,26,87,23,72,74,55,64,53
2,74,108,62,47,41,31,105,45,63,202,...,16,23,40,104,370,199,81,30,228,33
3,161,86,197,92,116,143,243,136,151,149,...,130,71,28,178,62,219,176,106,160,121
4,199,128,198,117,162,263,218,243,201,152,...,192,157,71,228,130,185,198,114,131,157


In [64]:
#Add gene id names to the raw data
gene_ids=data_df["ID"]

#double_precip = pd.concat([precip_one_station, precip_one_station])
raw_data.insert(0, "Gene_ID", gene_ids)
raw_data.head()

,Gene_ID,Raw_10_HC_Auto_066_237,Raw_11_Orkambi_006_Base,Raw_12_Orkambi_007_V2,Raw_13_HC_Auto_068_239,Raw_14_Orkambi_007_Base,Raw_15_Orkambi_009_V2,Raw_16_HC_Auto_072_243,Raw_17_Orkambi_009_Base,Raw_18_Orkambi_010_V2,...,Raw_Orkambi_021_Base,Raw_Orkambi_021_V2,Raw_Orkambi_022_Base,Raw_Orkambi_022_V2,Raw_Orkambi_024_Base,Raw_Orkambi_024_V2,Raw_Orkambi_025_Base,Raw_Orkambi_025_V2,Raw_Orkambi_026_Base,Raw_Orkambi_026_V2
0,A1BG,81,38,112,46,64,196,125,140,72,...,113,89,31,125,44,78,113,98,38,70
1,A1BG-AS1,49,31,84,26,38,86,80,76,38,...,40,36,26,87,23,72,74,55,64,53
2,A2M-AS1,74,108,62,47,41,31,105,45,63,...,16,23,40,104,370,199,81,30,228,33
3,AAAS,161,86,197,92,116,143,243,136,151,...,130,71,28,178,62,219,176,106,160,121
4,AACS,199,128,198,117,162,263,218,243,201,...,192,157,71,228,130,185,198,114,131,157


### Normalizing the data